<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Create and Configure a Vector Database to Store Document Embeddings**


Estimated time needed: **30** minutes


## Overview


Imagine you are working in a customer support center that receives a high volume of inquiries and tickets every day. Your task is to create a system that can quickly provide support agents with the most relevant information to resolve customer issues. Traditional methods of searching through FAQs or support documents can be slow and inefficient, leading to delayed responses and dissatisfied customers.

To address this challenge, you will use embedding models to convert support documents and past inquiry responses into numerical vectors that capture their semantic content. These vectors will be stored in a vector database, enabling fast and accurate similarity searches. For example, when a support agent receives a new inquiry about a product issue, the system can instantly retrieve similar past inquiries and their resolutions, helping the agent to provide a quicker and more accurate response.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/veZYoygp9GqZrIw5f6SD0g/vector%20db.png" width="50%" alt="vector db"/>


In this lab, you will learn how to use vector databases to store embeddings generated from textual data using LangChain. The focus will be on two popular vector databases: Chroma DB and FAISS (Facebook AI Similarity Search). You will also learn how to perform similarity searches in these databases based on a query, enabling efficient retrieval of relevant information. By the end of this lab, you will be able to effectively use vector databases to store and query embeddings, enhancing your data analysis and retrieval capabilities.


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Load-text">Load text</a></li>
            <li><a href="#Split-data">Split data</a></li>
            <li><a href="#Embedding model">Embedding model</a></li>
        </ol>
    </li>
    <li>
        <a href="#Vector-store">Vector store</a>
        <ol>
            <li><a href="#Chroma-DB">Chroma DB</a></li>
            <li><a href="#FIASS-DB">FIASS DB</a></li>
            <li><a href="#Managing-vector-store:-adding,-updating,-and-deleting-entries">Managing vector store: adding, updating, and deleting entries</a></li>
        </ol>
    </li>
</ol>

<a href="#Exercises">Exercises</a>
<ol>
    <li><a href="#Exercise-1---Use-another-query-to-conduct-similarity-search.">Exercise 1. Use another query to conduct similarity search.</a></li>
</ol>


## Objectives

After completing this lab you will be able to:

- Prepare and preprocess documents for embeddings.
- Generate embeddings using watsonx.ai's embedding model.
- Store these embeddings in Chroma DB and FAISS.
- Perform similarity searches to retrieve relevant documents based on new inquiries.


----


## Setup


For this lab, you will use the following libraries:

* [`ibm-watson-ai`](https://ibm.github.io/watsonx-ai-python-sdk/) for using LLMs from IBM's watsonx.ai.
* [`langchain`, `langchain-ibm`, `langchain-community`](https://www.langchain.com/) for using relevant features from Langchain.
* [`chromadb`](https://www.trychroma.com/) is a open-source vector database used to store embeddings.
* [`faiss-cpu`](https://pypi.org/project/faiss-cpu/) is used to support the using of FAISS vector database.


### Installing required libraries

The following required libraries are __not__ preinstalled in the Skills Network Labs environment. __You must run the following cell__ to install them:

**Note:** The version is being pinned here to specify the version. It's recommended that you do this as well. Even if the library is updated in the future, the installed library could still support this lab work.

This might take approximately 1-2 minutes. 

As `%%capture` is used to capture the installation, you won't see the output process. After the installation is completed, you will see a number beside the cell.


In [1]:
#%%capture
#!pip install "ibm-watsonx-ai==1.0.4"
#!pip install "langchain==0.2.1" 
#!pip install "langchain-ibm==0.1.7"
#!pip install "langchain-community==0.2.1"
#!pip install "chromadb==0.4.24"
#!pip install "faiss-cpu==1.8.0"

!pip install ibm-watsonx-ai
!pip install langchain 
!pip install langchain-ibm
!pip install langchain-community
!pip install chromadb
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 105.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 46.4 MB/s eta 0:00:00
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77230 sha256=e330a46dc51b2703a596b596c32b2056f44376d3345bd57

After you install the libraries, restart your kernel. You can do that by clicking the **Restart the kernel** icon.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/build-a-hotdog-not-hotdog-classifier-guided-project/images/Restarting_the_Kernel.png" width="50%" alt="Restart kernel">


-----


The following steps are prerequisite tasks for conducting this project's topic - vector store. These steps include:

- Loading the source document.
- Splitting the document into chunks.
- Building an embedding model.
  
The details of these steps have been introduced in previous lessons.


### Load text


A text file has been prepared as the source document for the downstream vector database task.

Now, let's download and load it using LangChain's `TextLoader`.


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BYlUHaillwM8EUItaIytHQ/companypolicies.txt"

--2024-11-22 04:33:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BYlUHaillwM8EUItaIytHQ/companypolicies.txt
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15660 (15K) [text/plain]
Saving to: ‘companypolicies.txt.2’

companypolicies.txt 100%[===================>]  15.29K  --.-KB/s    in 0s      

2024-11-22 04:33:35 (35.2 MB/s) - ‘companypolicies.txt.2’ saved [15660/15660]



In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader("companypolicies.txt")
data = loader.load()

You can have a look at this document.


In [5]:
data

[Document(metadata={'source': 'companypolicies.txt'}, page_content="1.\tCode of Conduct\n\nOur Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to 

### Split data


The next step is to split the document using LangChain's text splitter. Here, you will use the `RecursiveCharacterTextSplitter, which is well-suited for this generic text. The following parameters have been set:

- `chunk_size = 100`
- `chunk_overlap = 20`
- `length_function = len`


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

In [8]:
chunks = text_splitter.split_documents(data)

Let's take a look at how many chunks you get.


In [9]:
len(chunks)

215

So, in total, you get 215 chunks.


### Embedding model


The following code demonstrates how to build an embedding model using the `watsonx.ai` package.

For this project, the `ibm/slate-125m-english-rtrvr` embedding model will be used.


In [10]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 72.9 MB/s eta 0:00:00:00:0100:01


In [11]:
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params=embed_params,
)

The embedding model is formed into the `watsonx_embedding` object.


## Vector store


In this section, you will be guided on how to use two commonly used vector databases: Chroma DB and FAISS DB. You will also see how to perform a similarity search based on an input query using these databases.


### Chroma DB


#### Build the database


First, you need to import `Chroma` from Langchain vector stores.


In [12]:
from langchain.vectorstores import Chroma

Next, you need to create an ID list that will be used to assign each chunk a unique identifier, allowing you to track them later in the vector database. The length of this list should match the length of the chunks.

Note: The IDs should be in string format.


In [13]:
ids = [str(i) for i in range(0, len(chunks))]

The next step is to use the embedding model to create embeddings for each chunk and then store them in the Chroma database.

The following code demonstrates how to do this.


In [14]:
vectordb = Chroma.from_documents(chunks, watsonx_embedding, ids=ids)

Now that you have built the vector store named `vectordb`, you can use the method `.collection.get()` to print some of the chunks indexed by their IDs.

Note: Although the chunks are stored in the database in embedding format, when you retrieve and print them by their IDs, the database will return the chunk text information instead of the embedding vectors.


In [15]:
for i in range(3):
    print(vectordb._collection.get(ids=str(i)))

{'ids': ['0'], 'embeddings': None, 'documents': ['1.\tCode of Conduct'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
{'ids': ['1'], 'embeddings': None, 'documents': ['Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
{'ids': ['2'], 'embeddings': None, 'documents': ['that guide every member of our organization. We are committed to maintaining a workplace that is'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


You can also use the method `._collection.count()` to see the length of the vector database, which should be the same as the length of chunks.


In [16]:
vectordb._collection.count()

215

#### Similarity search


Similarity search in a vector database involves finding items that are most similar to a given query item based on their vector representations.

In this process, data objects are converted into vectors (which you've already done), and the search algorithm identifies and retrieves those with the closest vector distances to the query, enabling efficient and accurate identification of similar items in large datasets.


LangChain supports similarity search in vector stores using the method `.similarity_search()`.

The following is an example of how to perform a similarity search based on the query "Email policy."

By default, it will return the top four closest vectors to the query.


In [17]:
query = "Email policy"
docs = vectordb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='to this policy. Non-compliance may lead to appropriate disciplinary action, which could include'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

You can specify `k = 1` to just retrieve the top one result.


In [18]:
vectordb.similarity_search(query, k = 1)

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.')]

### FIASS DB


FIASS is another vector database that is supported by LangChain.

The process of building and using FAISS is similar to Chroma DB.

However, there may be differences in the retrieval results between FAISS and Chroma DB.


#### Build the database


Build the database and store the embeddings to the database here.


In [19]:
from langchain_community.vectorstores import FAISS

In [20]:
faissdb = FAISS.from_documents(chunks, watsonx_embedding, ids=ids)

Next, print the first three information pieces in the database based on IDs.


In [21]:
for i in range(3):
    print(vectordb._collection.get(ids=str(i)))

{'ids': ['0'], 'embeddings': None, 'documents': ['1.\tCode of Conduct'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
{'ids': ['1'], 'embeddings': None, 'documents': ['Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
{'ids': ['2'], 'embeddings': None, 'documents': ['that guide every member of our organization. We are committed to maintaining a workplace that is'], 'uris': None, 'data': None, 'metadatas': [{'source': 'companypolicies.txt'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


#### Similarity search


Let's do a similarity search again using FIASS DB on the same query.


In [22]:
query = "Email policy"
docs = faissdb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='to this policy. Non-compliance may lead to appropriate disciplinary action, which could include'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

The retrieve results based on the similarity search seem to be the same as with the Chroma DB.

You can try with other queries or documents to see if they follow the same situation.


### Managing vector store: Adding, updating, and deleting entries


There might be situations where new documents come into your RAG application that you want to add to the current vector database, or you might need to delete some existing documents from the database. Additionally, there may be updates to some of the documents in the database that require updating.

The following sections will guide you on how to perform these tasks. You will use the Chroma DB as an example.


#### Add


Imagine you have a new piece of text information that you want to add to the vector database. First, this information should be formatted into a document object.


In [23]:
text = "Instructlab is the best open source tool for fine-tuning a LLM."

In [24]:
from langchain_core.documents import Document

Form the text into a `Document` object named `new_chunk`.


In [25]:
new_chunk =  Document(
    page_content=text,
    metadata={
        "source": "ibm.com",
        "page": 1
    }
)

Then, the new chunk should be put into a list as the vector database only accepts documents in a list.


In [26]:
new_chunks = [new_chunk]

Before you add the document to the vector database, since there are 215 chunks with IDs from 0 to 214, if you print ID 215, the document should show no values. Let's validate it.


In [27]:
print(vectordb._collection.get(ids=['215']))

{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'data': None, 'metadatas': [], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


Next, you can use the method `.add_documents()` to add this `new_chunk`. In this method, you should assign an ID to the document. Since there are already IDs from 0 to 214, you can assign ID 215 to this document. The ID should be in string format and placed in a list.


In [28]:
vectordb.add_documents(
    new_chunks,
    ids=["215"]
)

['215']

Now you can count the length of the vector database again to see if it has increased by one.


In [29]:
vectordb._collection.count()

216

You can then print this newly added document from the database by its ID.


In [30]:
print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'documents': ['Instructlab is the best open source tool for fine-tuning a LLM.'], 'uris': None, 'data': None, 'metadatas': [{'page': 1, 'source': 'ibm.com'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


#### Update


Imagine you want to update the content of a document that is already stored in the database. The following code demonstrates how to do this.


Still, you need to form the updated text into a `Document` object.


In [31]:
update_chunk =  Document(
    page_content="Instructlab is a perfect open source tool for fine-tuning a LLM.",
    metadata={
        "source": "ibm.com",
        "page": 1
    }
)

Then, you can use the method `.update_document()` to update the specific stored information indexing by its ID.


In [32]:
vectordb.update_document(
    '215',
    update_chunk,
)

In [33]:
print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'documents': ['Instructlab is a perfect open source tool for fine-tuning a LLM.'], 'uris': None, 'data': None, 'metadatas': [{'page': 1, 'source': 'ibm.com'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


As you can see, the document information has been updated.


#### Delete


If you want to delete documents from the vector database, you can use the method `_collection.delete()` and specify the document ID to delete it.


In [34]:
vectordb._collection.delete(ids=['215'])

In [35]:
print(vectordb._collection.get(ids=['215']))

{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'data': None, 'metadatas': [], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


As you can see, now that document is empty.


# Exercises


### Exercise 1 - Use another query to conduct similarity search.

Can you use another query to conduct the similarity search?


In [36]:
# Your code here

query = "Smoking policy"
docs = vectordb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Designated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

## Authors


[Kang Wang](https://author.skills.network/instructors/kang_wang)

Kang Wang is a Data Scientist in IBM. He is also a PhD Candidate in the University of Waterloo.


### Other Contributors


[Joseph Santarcangelo](https://author.skills.network/instructors/joseph_santarcangelo)

Joseph has a Ph.D. in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


```{## Change Log}
```


```{|Date (YYYY-MM-DD)|Version|Changed By|Change Description||-|-|-|-||2024-07-24|0.1|Kang Wang|Create the lab|}
```



Copyright © IBM Corporation. All rights reserved.
